In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from string import punctuation
from random import sample

In [2]:

df1 = pd.read_csv("Data/Eclipse/EP_dup.csv", sep=";")
df2 = pd.read_csv("Data/Eclipse/EP_nondup.csv", sep=";")
df3 = pd.read_csv("Data/Mozilla/M_Duplicate BRs.csv", sep=";")  
df4 = pd.read_csv("Data/Mozilla/M_NonDuplicate BRs.csv", sep=";")
df5 = pd.read_csv("Data/ThunderBird/dup_TB.csv", sep=";")
df6 = pd.read_csv("Data/ThunderBird/Nondup_TB.csv", sep=";")

frames=[df1,df2,df3,df4,df5,df6]
data=pd.concat(frames)
data

,Issue_id,Duplicated_issue,Title1,Description1,Title2,Description2,Label
0,25,28126,cvs ui need vcm prefs default repo connection gc,it would be helpful if there was a notion of d...,wizards patch standard public cvs repositories,this patch adds a convenient way to check thin...,1
1,40,20,need connect to team stream gcqpkw,i would like to be able to connect to a team s...,workspace files,thought it would be useful if the set of repo ...,1
2,48,22,make sure can future store other project refer...,project references come in three flavours . p...,persist sharing recommendations and project ve...,project descriptions dont store sharing recomm...,1
3,61,60,.vcmmeta showing as change gdqtgw,useruser install drop into declipse user ...,need custom .vcmignore comparemerge gdqt,useruser install drop into declipse user ...,1
4,94,2,repositories view all file types open to the t...,when browsing files in the repositories view i...,opening repository resources doesnt honor type...,opening repository resource open the default ...,1
...,...,...,...,...,...,...,...
9900,418747,953052,thunderbird outlook migrate lost outlooks pst ...,i installed thunderbird hoping i could impor...,i am using thunderbird esr latest but i keep ...,i am using thunderbird .. esr latest but i...,0
9901,418821,953165,urls in mail with comma opens with c in browser,when clicking in text mail not html on url w...,picture in feed not showing,. i linked the rssfeed from httpwww.nu.nlf...,0
9902,418832,953249,greek attachments not displayed correctly,i am using gmail imap accounts. when i recei...,add the flagcolpng to linux,httpmxr.mozilla.orgcommcentralsearchstringflag...,0
9903,419035,953375,branch only random crash when i want to displ...,the problem appears with icedove the debian ...,replace undefined gpromptservice with services...,patch v discovered a call to undefined gprom...,0


In [4]:
data = data.reset_index()
data


,level_0,index,Issue_id,Duplicated_issue,Title1,Description1,Title2,Description2,Label
0,0,0,25,28126,cvs ui need vcm prefs default repo connection gc,it would be helpful if there was a notion of d...,wizards patch standard public cvs repositories,this patch adds a convenient way to check thin...,1
1,1,1,40,20,need connect to team stream gcqpkw,i would like to be able to connect to a team s...,workspace files,thought it would be useful if the set of repo ...,1
2,2,2,48,22,make sure can future store other project refer...,project references come in three flavours . p...,persist sharing recommendations and project ve...,project descriptions dont store sharing recomm...,1
3,3,3,61,60,.vcmmeta showing as change gdqtgw,useruser install drop into declipse user ...,need custom .vcmignore comparemerge gdqt,useruser install drop into declipse user ...,1
4,4,4,94,2,repositories view all file types open to the t...,when browsing files in the repositories view i...,opening repository resources doesnt honor type...,opening repository resource open the default ...,1
...,...,...,...,...,...,...,...,...,...
122070,122070,9900,418747,953052,thunderbird outlook migrate lost outlooks pst ...,i installed thunderbird hoping i could impor...,i am using thunderbird esr latest but i keep ...,i am using thunderbird .. esr latest but i...,0
122071,122071,9901,418821,953165,urls in mail with comma opens with c in browser,when clicking in text mail not html on url w...,picture in feed not showing,. i linked the rssfeed from httpwww.nu.nlf...,0
122072,122072,9902,418832,953249,greek attachments not displayed correctly,i am using gmail imap accounts. when i recei...,add the flagcolpng to linux,httpmxr.mozilla.orgcommcentralsearchstringflag...,0
122073,122073,9903,419035,953375,branch only random crash when i want to displ...,the problem appears with icedove the debian ...,replace undefined gpromptservice with services...,patch v discovered a call to undefined gprom...,0


In [5]:


data['TitleDescription1'] = data['Title1'].str.cat(data['Description1'],sep=" ")
data['TitleDescription2'] = data['Title2'].str.cat(data['Description2'],sep=" ")


In [6]:
atitle = data['Title1'].str.len()

In [7]:
bdes = data['Description1'].str.len()
ccom1 = data['TitleDescription1'].str.len()
ccom2 = data['TitleDescription2'].str.len()

In [9]:
bdes1 = []
for i in bdes:
        if i>200:
            bdes1.append(i)
atitle1 = []
for i in atitle:
        if i>40:
            atitle1.append(i)
            
            
            
            


In [102]:
ccom2

0          312
1          452
2          266
3         1678
4         1029
          ... 
122070     304
122071     543
122072     384
122073     139
122074     529
Name: TitleDescription2, Length: 122075, dtype: int64

In [103]:
ccom11 = []
for i in ccom1:
        if i>200:
            ccom11.append(data.index[i])
ccom21 = []
for i in ccom2:
        if i>10:
            ccom21.append(i)

In [115]:
try1 = pd.DataFrame()
try1['Label'] = data['TitleDescription1'].str.len()



0

In [116]:
try2 = pd.DataFrame()
for i in try1.index:
        if try1['Label'][i]>200:
            try2 = try2.append(data.iloc[i])

MemoryError: Unable to allocate 3.12 MiB for an array with shape (6, 68151) and data type object

In [16]:
fin1 = []
for i in ccom11:
    if i in ccom21:
        fin1.append(i)

In [17]:
len(fin1)

108087

In [18]:
data1 = pd.DataFrame()
data1['Bug1'] = data['TitleDescription1']
data1['Bug2'] = data['TitleDescription2']
data1['Label'] = data['Label']


In [19]:
data1

,Bug1,Bug2,Label
0,cvs ui need vcm prefs default repo connection...,wizards patch standard public cvs repositories...,1
1,need connect to team stream gcqpkw i would lik...,workspace files thought it would be useful if ...,1
2,make sure can future store other project refer...,persist sharing recommendations and project ve...,1
3,.vcmmeta showing as change gdqtgw useruser i...,need custom .vcmignore comparemerge gdqt useru...,1
4,repositories view all file types open to the t...,opening repository resources doesnt honor type...,1
...,...,...,...
122070,thunderbird outlook migrate lost outlooks pst ...,i am using thunderbird esr latest but i keep ...,0
122071,urls in mail with comma opens with c in brows...,picture in feed not showing . i linked the...,0
122072,greek attachments not displayed correctly i ...,add the flagcolpng to linux httpmxr.mozilla.or...,0
122073,branch only random crash when i want to displ...,replace undefined gpromptservice with services...,0


In [20]:
data3 = pd.DataFrame()
for i in ccom11:
        data3 = data3.append(data1.iloc[i])

In [21]:
len(data3)

110243

In [22]:
data3 = data3.reset_index()
data3

,index,Bug1,Bug2,Label
0,483,sturcture compare changes selected in java str...,move right to left misplaced the moved text us...,1.0
1,745,stringindexoutofboundsexception trying to clos...,stringindexoutofboundsexception opening java f...,1.0
2,803,.doc file open failed when i creat a word docu...,unable to add word document to project version...,1.0
3,1344,gtk selectionevent.detail does not return expe...,selectionevent not getting detail info for scr...,1.0
4,1343,resources resource out of synch update confli...,resources build resource out of sync error th...,1.0
...,...,...,...,...
110238,867,f help flashes and disappears ths mght be a du...,javadoc hover never receives focusevents i on ...,1.0
110239,452,internal error dialog prompts yesno but doesnt...,incomplete message in emergency shutdown dialo...,1.0
110240,1008,toolbar layout has big space in middle i have ...,editoractionbarcontributor toolbar actions way...,1.0
110241,1020,sizing problems of the launch config dialog st...,not all tabs visible in runtime workbench conf...,1.0


In [23]:
data1['Label'].value_counts()

0    80960
1    41115
Name: Label, dtype: int64

In [24]:
data3 = data3.drop('Label', axis=1)

In [25]:
data3

,index,Bug1,Bug2
0,483,sturcture compare changes selected in java str...,move right to left misplaced the moved text us...
1,745,stringindexoutofboundsexception trying to clos...,stringindexoutofboundsexception opening java f...
2,803,.doc file open failed when i creat a word docu...,unable to add word document to project version...
3,1344,gtk selectionevent.detail does not return expe...,selectionevent not getting detail info for scr...
4,1343,resources resource out of synch update confli...,resources build resource out of sync error th...
...,...,...,...
110238,867,f help flashes and disappears ths mght be a du...,javadoc hover never receives focusevents i on ...
110239,452,internal error dialog prompts yesno but doesnt...,incomplete message in emergency shutdown dialo...
110240,1008,toolbar layout has big space in middle i have ...,editoractionbarcontributor toolbar actions way...
110241,1020,sizing problems of the launch config dialog st...,not all tabs visible in runtime workbench conf...


In [26]:
data4 = pd.DataFrame()
data4['Label'] = data['Label']
data4

,Label
0,1
1,1
2,1
3,1
4,1
...,...
122070,0
122071,0
122072,0
122073,0


In [81]:
data5 = pd.DataFrame()
data5['Label'] = data['Label']
data5

,Label
0,1
1,1
2,1
3,1
4,1
...,...
122070,0
122071,0
122072,0
122073,0


In [94]:
data6 = pd.DataFrame()
not1 = []
#for i in data.index:
#    if i not in ccom11:
#        data6 = data1.drop(data1.index[i])
for i in data.index:
    if i not in ccom11:
        not1.append(i)

            

In [98]:
len(data)-len(not1)

5224

In [97]:
len(ccom11)

110243

In [ ]:
len(data6)

In [ ]:
data6

In [ ]:
type(data['Label'][0])

In [ ]:
type(data6['Label'][483])

In [229]:
stop_words = set(stopwords.words('english'))

In [230]:
def words(text, remove_stop_words=True, stem_words=False):
    # Remove punctuation from questions
    text = ''.join([c for c in text if c not in punctuation])
    
    # Lowering the words in questions
    text = text.lower()
    
    # Remove stop words from questions
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Return a list of words
    return(text)

In [231]:
def process(bug_list, bugs):
    for bug in bugs:
        bug_list.append(words(bug))
processed_bug1 = []
processed_bug2 = []
process(processed_bug1, data3.Bug1)
process(processed_bug2, data3.Bug2)

In [232]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer = 'word',
                        stop_words = 'english',
                        lowercase = True,
                        max_features = 300,
                        norm = 'l1')

In [233]:
words = pd.concat([data3.Bug1, data3.Bug2], axis = 0)

In [234]:
tfidf.fit(words)
duplicate_1 = tfidf.transform(data3.Bug1)
duplicate_2 = tfidf.transform(data3.Bug2)

In [235]:
x = abs(duplicate_1 - duplicate_2)
y = data3['Label']

In [236]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [237]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv1D, GlobalMaxPooling1D


model = Sequential()
model.add(Dense(64,input_shape=(300,)))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 64)                19264     
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_15 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_16 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                

In [238]:
results = model.fit(x_train.todense(),y_train,batch_size=64,epochs=5,verbose=1,validation_data=(x_test.todense(),y_test))


Epoch 1/5
1499/1499 [==============================] - 4s 3ms/step - loss: 0.0182 - acc: 0.9994 - val_loss: 0.0016 - val_acc: 0.9999
Epoch 2/5
1499/1499 [==============================] - 3s 2ms/step - loss: 0.0025 - acc: 0.9998 - val_loss: 0.0015 - val_acc: 0.9999
Epoch 3/5
1499/1499 [==============================] - 3s 2ms/step - loss: 0.0021 - acc: 0.9998 - val_loss: 0.0015 - val_acc: 0.9999
Epoch 4/5
1499/1499 [==============================] - 4s 3ms/step - loss: 0.0019 - acc: 0.9998 - val_loss: 0.0016 - val_acc: 0.9999
Epoch 5/5
1499/1499 [==============================] - 3s 2ms/step - loss: 0.0015 - acc: 0.9998 - val_loss: 0.0018 - val_acc: 0.9999


In [239]:
print(np.mean(results.history["val_acc"]))

0.9998748898506165


In [240]:
from sklearn.metrics import classification_report
pred = model.predict_classes(x_test)
print(classification_report(y_test,pred,labels=[1,0]))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     23981
           0       0.00      0.00      0.00         3

    accuracy                           1.00     23984
   macro avg       0.50      0.50      0.50     23984
weighted avg       1.00      1.00      1.00     23984



C:\Users\sawag\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sawag\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sawag\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [241]:
results